# Prize-Collecting Steiner Tree (PCSTP)

## Libs Importing

In [1]:
import sys
import os
import time
import networkx as nx

sys.path.insert(1, os.path.realpath(os.path.pardir))

In [2]:
import multiprocessing

NUM_PROCESSES = multiprocessing.cpu_count()
print("Number of cpu : ", NUM_PROCESSES)

Number of cpu :  12


In [3]:
from pcstp.instances.generator import generate_random_steiner
from pcstp.instances.reader import SteinlibReader, DatReader

from pcstp.steinertree import SteinerTreeProblem
from pcstp.solver.base import computes_steiner_cost
from pcstp.solver.aco import AntColony
from pcstp.solver.greedy_h1 import GreedyH1

from pcstp.utils.graph import preprocessing
from pcstp.utils.draw import draw_steiner_graph

## Experiments

In [ ]:
SEED = 100

In [ ]:
import glob

INSTANCES_PATH_PREFIX = '../data/instances/benchmark/PCSPG-CRR'
NUM_EXPERIMENTS_PER_INSTANCE = 10

all_files = glob.glob(os.path.join(INSTANCES_PATH_PREFIX, '*'))

files = all_files

networkx_history = []

for filename in files:
    if filename.endswith('.xlsx'): continue
    if filename.endswith('.stp'):
        stp_reader = SteinlibReader()
    else:
        stp_reader = DatReader()

    print(f"Reading: {filename}")
    stp = stp_reader.parser(filename=filename)
    G, terminals = preprocessing(stp.graph, stp.terminals)
    stp_preprocessed = SteinerTreeProblem(graph=G, terminals=terminals)
    
    def run_experiment(experiment: int):
        start_time = time.time()
        nx_steiner_tree = nx.algorithms.approximation.steiner_tree(
            stp_preprocessed.graph,
            stp_preprocessed.terminals,
            weight='cost'
        )

        networkx_duration = time.time() - start_time
        networkx_cost = computes_steiner_cost(stp.graph, nx_steiner_tree, stp.terminals)

        history = {
            "filename": filename,
            "experiment": experiment,
            "num_nodes": stp.num_nodes,
            "num_edges": stp.num_edges,
            "num_nodes_after_preprocessing": len(stp_preprocessed.graph.nodes),
            "num_edges_after_preprocessing": len(stp_preprocessed.graph.edges),
            "terminals": stp.num_terminals,
            "steiner_cost": networkx_cost,
            "duration": networkx_duration
        }
        return history

    experiments = range(1, NUM_EXPERIMENTS_PER_INSTANCE+1)

    with multiprocessing.Pool(processes=NUM_PROCESSES) as p:
        experiments_results = p.map(run_experiment, experiments)
    
    networkx_history.extend(experiments_results)

In [ ]:
import pandas as pd

df_score_networkx = pd.DataFrame.from_dict(networkx_history)

In [ ]:
df_score_networkx.groupby('filename')[['duration', 'steiner_cost']].describe()


# Greedy

In [ ]:
import glob
import random
import numpy as np

INSTANCES_PATH_PREFIX = '../data/instances/benchmark/PCSPG-CRR'
NUM_EXPERIMENTS_PER_INSTANCE = 5

all_files = glob.glob(os.path.join(INSTANCES_PATH_PREFIX, '*'))

files = all_files

greedy_history = []

for filename in files:
    if filename.endswith('.xlsx'): continue
    if filename.endswith('.stp'):
        stp_reader = SteinlibReader()
    else:
        stp_reader = DatReader()

    print(f"Reading: {filename}")
    stp = stp_reader.parser(filename=filename)
    G, terminals = preprocessing(stp.graph, stp.terminals)
    stp_preprocessed = SteinerTreeProblem(graph=G, terminals=terminals)
    print("Nodes: ", len(stp_preprocessed.graph.nodes))
    print("Edges: ", len(stp_preprocessed.graph.edges))
    # print("Terminals: ", stp_preprocessed.terminals)
    def run_experiment(experiment: int):
        if SEED:
            np.random.seed(SEED*experiment)
            random.seed(SEED*experiment)
        solver = GreedyH1(stp_preprocessed.graph, list(stp_preprocessed.terminals), log_level='info')
        steiner_tree, greedy_cost = solver.solve()
        print(f'Cost: {greedy_cost} ')

        history = {
            "filename": filename,
            "experiment": experiment,
            "num_nodes": stp.num_nodes,
            "num_edges": stp.num_edges,
            "num_nodes_after_preprocessing": len(stp_preprocessed.graph.nodes),
            "num_edges_after_preprocessing": len(stp_preprocessed.graph.edges),
            "terminals": stp.num_terminals,
            "steiner_cost": greedy_cost,
            "duration": solver._duration
        }
        return history

    experiments = range(1, NUM_EXPERIMENTS_PER_INSTANCE+1)

    with multiprocessing.Pool(processes=NUM_PROCESSES) as p:
        experiments_results = p.map(run_experiment, experiments)

    greedy_history.extend(experiments_results)


In [ ]:
import pandas as pd

df_score_greedy = pd.DataFrame.from_dict(greedy_history)

## Solution obtained with Ant Colony Optimization

In [ ]:
import glob

INSTANCES_PATH_PREFIX = '../data/instances/benchmark/PCSPG-CRR'
NUM_EXPERIMENTS_PER_INSTANCE = 5

all_files = glob.glob(os.path.join(INSTANCES_PATH_PREFIX, '*'))

files = all_files

aco_history = []

for filename in files:
    if filename.endswith('.stp'):
        stp_reader = SteinlibReader()
    else:
        stp_reader = DatReader()

    print(f"Reading: {filename}")
    stp = stp_reader.parser(filename=filename)
    G, terminals = preprocessing(stp.graph, stp.terminals)
    stp_preprocessed = SteinerTreeProblem(graph=G, terminals=terminals)

    def run_experiment(experiment: int):
        aco_params = dict(
            iterations=1,
            num_ants=len(terminals),
            evaporation_rate=0.5,
            alpha=1.0,
            beta=3.0,
            # beta_evaporation_rate=0.2,
            initial_pheromone=0.1,
            pheromone_amount=2.0,
            pheromone_deposit_strategy='traditional',
            pheromone_initialization_strategy='same_value',
            choose_best=0.2,
            log_level='info',
            early_stopping=30,
            normalize_distance_prize=False,
            allow_edge_perturbation=False,
            ant_max_moves=1,
            seed=SEED * experiment
        )
        solver = AntColony(
            graph=stp_preprocessed.graph,
            terminals=stp_preprocessed.terminals,
            **aco_params
        )
        steiner_tree, steiner_cost = solver.solve()

        history = {
            "filename": filename,
            "experiment": experiment,
            "num_nodes": stp.num_nodes,
            "num_edges": stp.num_edges,
            "num_nodes_after_preprocessing": len(stp_preprocessed.graph.nodes),
            "num_edges_after_preprocessing": len(stp_preprocessed.graph.edges),
            "terminals": stp.num_terminals,
            "steiner_cost": steiner_cost,
            "duration": solver._duration
        }
        history.update(aco_params)
        return history

    experiments = range(1, NUM_EXPERIMENTS_PER_INSTANCE+1)

    with multiprocessing.Pool(processes=NUM_PROCESSES) as p:
        experiments_results = p.map(run_experiment, experiments)
    
    aco_history.extend(experiments_results)

In [ ]:
import pandas as pd

df_score_aco = pd.DataFrame.from_dict(networkx_history)
# df_score_aco.to_csv(os.path.join(INSTANCES_PATH_PREFIX, 'ACO.csv'))